In [1]:
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### Data is downloaded from: https://grouplens.org/datasets/movielens/https://grouplens.org/datasets/movielens/

In [2]:
nRowsRead = None # specify 'None' if want to read whole file
# movie_metadata.csv has 5044 rows in reality, but we are only loading/previewing the first 1000 rows
movies = pd.read_csv('../data/movie-lens-small/movies.csv', delimiter=',', nrows = nRowsRead)
ratings = pd.read_csv('../data/movie-lens-small/ratings.csv', delimiter=',', nrows = nRowsRead)
tags = pd.read_csv('../data/movie-lens-small/tags.csv', delimiter=',', nrows = nRowsRead)

In [3]:
movies.shape

(9742, 3)

Let's take a quick look at what the data looks like:

In [4]:
pd.set_option('display.max_columns', None)

display(movies.head(5))
display(ratings.head(5))
display(tags.head(5))


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


### first time setup
- pipenv install "elasticsearch<8.0.0"
- docker pull docker.elastic.co/elasticsearch/elasticsearch:7.10.0
- docker run --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.10.0

### second time and beyond
- docker stop elasticsearch
- docker start elasticsearch

In [21]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch with HTTP (not HTTPS)
es = Elasticsearch(
    ['http://localhost:9200']  # Use HTTP instead of HTTPS
)

# Define the index name
index_name = 'movies_index'

# Check if the index exists
if es.indices.exists(index=index_name):
    # Delete the index
    es.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted.")
else:
    print(f"Index '{index_name}' does not exist.")

# Create the index
print(f"Index '{index_name}' creating...")

es.indices.create(index=index_name, ignore=400)

# Indexing documents from the dataframe
for i, row in movies.iterrows():
    doc = {
        'title': row['title'],
        'movieId': row['movieId'],
        'genres': row['genres']
    }
    if i % 500 == 0:
        print(f'indexing the {i}th docs')
    es.index(index=index_name, document=doc)
print(f"Index '{index_name}' created.")

Index 'movies_index' deleted.
Index 'movies_index' creating...
indexing the 0th docs
indexing the 500th docs
indexing the 1000th docs
indexing the 1500th docs
indexing the 2000th docs
indexing the 2500th docs
indexing the 3000th docs
indexing the 3500th docs
indexing the 4000th docs
indexing the 4500th docs
indexing the 5000th docs
indexing the 5500th docs
indexing the 6000th docs
indexing the 6500th docs
indexing the 7000th docs
indexing the 7500th docs
indexing the 8000th docs
indexing the 8500th docs
indexing the 9000th docs
indexing the 9500th docs
Index 'movies_index' created.


In [23]:
def search_results_es(query_text, es, index_name, top_k):
    query = {
        "multi_match": {
            "query": query_text,
            "fields": ["title^2", "genres^0.5"]
        }
    }
    results = es.search(index=index_name, query=query, size=top_k)
    return [(hit['_source'].get('title'), hit['_source'].get('movieId'), hit['_source'].get('genres')) for hit in results['hits']['hits']]


In [25]:
# Example search
query_text = "toy"
search_results = search_results_es(query_text, es, index_name, top_k=10)
print(search_results)

[('Toy Soldiers (1991)', 5843, 'Action|Drama'), ('Toy, The (1982)', 4929, 'Comedy'), ('Toy Story (1995)', 1, 'Adventure|Animation|Children|Comedy|Fantasy'), ('Toy Story 2 (1999)', 3114, 'Adventure|Animation|Children|Comedy|Fantasy'), ('Toy Story 3 (2010)', 78499, 'Adventure|Animation|Children|Comedy|Fantasy|IMAX')]


# Load model 

In [26]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch with HTTP (not HTTPS)
es = Elasticsearch(
    ['http://localhost:9200']  # Use HTTP instead of HTTPS
)

# Define the index name
index_name = 'movies_index'


In [27]:
def search_results_es(query_text, es, index_name, top_k):
    query = {
        "multi_match": {
            "query": query_text,
            "fields": ["title^2", "genres^0.5"]
        }
    }
    results = es.search(index=index_name, query=query, size=top_k)
    return [(hit['_source'].get('title'), hit['_source'].get('movieId'), hit['_source'].get('genres')) for hit in results['hits']['hits']]


In [28]:
# Example search
query_text = "toy"
search_results = search_results_es(query_text, es, index_name, top_k=10)
print(search_results)

[('Toy Soldiers (1991)', 5843, 'Action|Drama'), ('Toy, The (1982)', 4929, 'Comedy'), ('Toy Story (1995)', 1, 'Adventure|Animation|Children|Comedy|Fantasy'), ('Toy Story 2 (1999)', 3114, 'Adventure|Animation|Children|Comedy|Fantasy'), ('Toy Story 3 (2010)', 78499, 'Adventure|Animation|Children|Comedy|Fantasy|IMAX')]
